In [1]:
import torch
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
test_df = pd.read_json('test.jsonl', lines=True)
X_test = test_df['target_text']
y_test = test_df['annotation']

In [6]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

#Load the pre-trained model
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def roberta_polarity_scores(text):
    encoded_text = tokenizer(text, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'negative': scores[0],
        'neutral': scores[1],
        'positive': scores[2]
    }
    return scores_dict

def roberta_sentiment(text):
    try:
        scores_dict = roberta_polarity_scores(text)
        sentiment = max(scores_dict, key=scores_dict.get)
        return sentiment
    except:
        return ""

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
labels = ['negative', 'neutral', 'positive', 'unrelated']

test_df['roberta_sentiment'] = test_df['target_text'].apply(roberta_sentiment)
y_pred = test_df['roberta_sentiment']
print("test accuracy: %.2f%%" % (accuracy_score(y_pred, y_test)*100))
print(classification_report(y_test, y_pred, target_names=labels))

test accuracy: 34.68%
              precision    recall  f1-score   support

    negative       0.52      0.73      0.61       127
     neutral       0.12      0.74      0.20        23
    positive       0.36      0.67      0.47       129
   unrelated       0.00      0.00      0.00       289

    accuracy                           0.35       568
   macro avg       0.25      0.54      0.32       568
weighted avg       0.20      0.35      0.25       568

